# Flow accumulation D8
The D8 flow accumulation method, also known as the "steepest descent" method, is a common approach used in hydrology and geomorphology to model surface water flow over a digital elevation model (DEM). In this method, flow is assumed to follow the steepest downward gradient from each cell to one of its eight neighboring cells.

Here's how the D8 flow accumulation method works:

- **Determination of Flow Direction**: For each cell in the DEM, the steepest downward gradient is determined by comparing the elevation of the cell with the elevations of its eight neighboring cells. The neighboring cell with the lowest elevation is considered the downstream cell, and flow is directed towards it.

- **Flow Accumulation**: Flow accumulation is calculated by summing up the flow coming from all upstream cells. Starting from the cells with the lowest elevation (the "pour points"), flow is accumulated downstream. As flow moves downstream, it accumulates contributions from upstream cells. Cells with higher flow accumulation values are indicative of areas that potentially contribute more water to the downstream flow.

- **Flow Routing**: Once the flow direction and accumulation are determined, it is possible to model the flow path of water across the entire DEM. This information is valuable for various hydrological analyses, such as predicting runoff patterns, identifying drainage networks, and assessing flood risks.

The D8 method assumes that water flows only in the direction of steepest descent, without considering factors such as soil type, vegetation, or land use, which can influence actual flow patterns. While the D8 method is simple and computationally efficient, it may oversimplify the flow process in complex terrains, particularly in areas with flat or ambiguous topography, leading to potential inaccuracies in flow routing and accumulation estimates.

## 1. Import libraries

In [1]:
import numpy as np
import plotly.express as px
import rasterio
from ipywidgets import widgets
import os

## Flow accumlation - D8
This function computes the flow contribution to neighbouring cells based on the slope from the current cell (x, y). It calculates the slope in eight directions using the DEM and normalizes these slopes to get flow contributions. The function returns an array containing these flow contributions.

In [2]:
def direction_lowest_neighbour(x, y, dem):
    """
    Helper function to determine the direction to the lowest neighboring cell.

    Args:
        x, y (int): Coordinates of the cell.
        dem (numpy.ndarray): A 2D array representing the elevation data.

    Returns:
        dx, dy, flow_dir (tuple): The direction to the lowest neighbour (dx, dy) and the corresponding flow direction.
    """
    
    rows, cols = dem.shape  # Get the dimensions of the DEM (rows, cols)
    
    # Define the neighbourhood directions (8-connected)
    neighbours = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),           (0, 1),
                  (1, -1),  (1, 0),  (1, 1)]
    
    # The flow direction is encoded as integer values ranging from 1 to 255 (GIS format) 
    # Each direction corresponds to a specific bit in the binary representation of the flow direction.
    # The values for each direction from the center are as follows:
    # 32 | 64 | 128
    # 16 |    |   1
    #  8 |  4 |   2
    neighbours_dir = [32, 64, 128,
                      16,       1,
                       8,  4,   2]
    
    neighbour_elevations = []

    for dx, dy in neighbours:
        nx, ny = x + dx, y + dy
        # Check if neighbour is within bounds
        if 0 <= nx < rows and 0 <= ny < cols:
            neighbour_elevations.append(dem[nx, ny])
        else:
            neighbour_elevations.append(np.nan)

    # Determine the direction to the lowest neighbour based on the minimum elevation
    dx, dy = neighbours[np.nanargmin(neighbour_elevations)]
    
    # Determine the flow direction based on the direction to the lowest neighbour
    flow_dir = neighbours_dir[np.nanargmin(neighbour_elevations)]

    return dx, dy, flow_dir

This function calculates flow accumulation using the D8 method. It iterates through each non-NaN cell, computes flow contributions to its neighbours, updates flow accumulation accordingly, and stores upstream and downstream cell indices along with flow contributions. Finally, it returns the flow accumulation matrix, and upstream and downstream indices.

In [3]:
def flow_accumulation_D8(dem):
    """
    Calculates the flow accumulation for a digital elevation model (DEM).

    Args:
        dem (numpy.ndarray): A 2D array representing the elevation data.

    Returns:
        - fa (numpy.ndarray): The flow accumulation values for each cell.
        - ix (numpy.ndarray): The upstream cell index for each cell (not including source cells).
        - ixc (numpy.ndarray): The downstream cell index for each cell (not including sink cells).
    """

    # Create a copy of DEM to work with
    dem_temp = dem.copy()
    
    # Initialize the flow accumulation matrix with ones
    fa = np.ones_like(dem, dtype=float)
    fa[np.isnan(dem)] = np.nan
    
    # Initialize the flow direction matrix with zeros
    fd = np.zeros_like(dem, dtype=float)
    fd[np.isnan(dem)] = np.nan

    # Calculate the total number of non-NaN cells
    num_nonan = np.sum(~np.isnan(dem))

    # Initialize arrays to store upstream and downstream cell indices
    ix  = np.zeros(num_nonan, dtype=int)
    ixc = np.zeros(num_nonan, dtype=int)
    
    # Iterate through all non-NaN cells
    for i in range(num_nonan-1):

        # Find the highest remaining cell in the temporary DEM
        high_cell_index = np.nanargmax(dem_temp)
        high_cell = np.unravel_index(high_cell_index, dem_temp.shape)
        # Another more math-like way
        #high_cell = np.where(dem_temp == np.nanmax(dem_temp))[0][0], np.where(dem_temp == np.nanmax(dem_temp))[1][0]
        
        # Calculate the direction to the lowest neighbour (D8)
        dx, dy, fd[high_cell] = direction_lowest_neighbour(high_cell[0],high_cell[1], dem)

        # Increment flow accumulation of the lowest neighbour
        fa[high_cell[0] + dx, high_cell[1] + dy] += fa[high_cell]

        # Store indices of highest cell and lowest neighbouring cell, which define the flow direction
        ix[i]  = high_cell_index
        ixc[i] = high_cell_index + dx * dem.shape[1] + dy

        # Mark the processed cell (highest cell) as inactive in the temporary DEM
        dem_temp[high_cell[0],high_cell[1]] = np.nan

        # Optional: Using a flow accumulation threshold, remove the pixels of the gully (uncomment if needed)
        #fa[fa>1000] = np.nan

    return fd, fa, ix, ixc

### Load the filled DEM of the catchment of study
We use `rasterio` to manipulate raster datasets, extract information from them, and perform various raster operations.

In [4]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]
dem_files = []
for s in files:
    if 'asc' in s and 'dem' in s and 'fill' in s:
        dem_files.append(s)
input_file = widgets.Select(
    options=dem_files,
    description='dem files:',
    disabled=False
)
display(input_file)

Select(description='dem files:', options=('dem_fill.asc', 'dem_syn_fill.asc'), value='dem_fill.asc')

In [68]:
# Open the raster map file
with rasterio.open(input_file.value) as src:
    # Read the raster data as a numpy array
    dem = src.read(1)  # Read the first band (index 0)

    # Get metadata of the raster map
    dem_metadata = src.meta

The `src.read(1)` function call reads the raster data as a numpy array. The `metadata` variable contains metadata information such as the raster's spatial reference system, data type, and geotransform. You can use this metadata for various purposes, such as georeferencing and understanding the properties of the raster map.

### Run the function to compute the flow accumulation using Dinf

In [69]:
fd_D8,fa_D8,ix_D8,ixc_D8 = flow_accumulation_D8(dem)

fig = px.imshow(fd_D8,color_continuous_scale='RdBu_r')
fig.show()

In [70]:
fig = px.imshow(fa_D8,color_continuous_scale='RdBu_r')
fig.show()

In [71]:
fig = px.imshow(dem,color_continuous_scale='RdBu_r')
fig.show()

### Save the flow direction as a raster file

In [8]:
if 'syn' in input_file.value:
    # Specify the output file path
    output_file = 'fd_D8_syn.asc'    
else:
    # Specify the output file path
    output_file = 'fd_D8.asc'

# Write the modified raster data to a new file with the same metadata
with rasterio.open(output_file, 'w', **dem_metadata) as dst:
    # Write the modified raster data to the new file
    dst.write(fd_D8, 1)  # Assuming raster_data is the modified array

C:\Users\Usuario\anaconda3\envs\iMPACt\lib\site-packages\rasterio\__init__.py:341: NotGeoreferencedWarning:

The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.



### Save the flow accumulation as a raster file

In [9]:
if 'syn' in input_file.value:
    # Specify the output file path
    output_file = 'fa_D8_syn.asc'    
else:
    # Specify the output file path
    output_file = 'fa_D8.asc'

# Write the modified raster data to a new file with the same metadata
with rasterio.open(output_file, 'w', **dem_metadata) as dst:
    # Write the modified raster data to the new file
    dst.write(fa_D8, 1)  # Assuming raster_data is the modified array